In [1]:
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from textScrapper import TextScrapper
from tqdm import tqdm
import numpy as np
import os
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
import string
import re
import warnings
from sklearn.model_selection import train_test_split
warnings.filterwarnings('ignore')

In [2]:
# exemple path file ../data/sncf_vinci_ifg_62/00086570-002601E-B007-20220323-R-VEN.pdf
class fileTextManager():
    
    def __init__ (self):
        self.current_DB= "sncf_vinci_ifg_62"
        
        
    def cleanText(self, text):
        char="abcdefghijklmnopqrstuvwxyzéèàçôùëïüûâùäîê-. "
        lowercaseText= text[0].lower()
        cleanText = ""
        for letter in lowercaseText:
            if letter in char:
                cleanText = cleanText + letter
        return cleanText

    def getExtension(self, file):
        filename, file_extension = os.path.splitext(file)
        return file_extension
    
    
    def getFileText(self, file):
        textScrap = TextScrapper()
        typeDocumentNumber=[]
        match file["mimeType"]:
            case 'application/pdf':
                try:
                    fullText = textScrap.getTextPdf(f"/Volumes/DD Thibault/sncf_DB/doc/{self.current_DB}/{file['KidFile']}")
                except:
                    fullText = None
                fullText = self.cleanText(fullText)
                return fullText
            case 'application/vnd.ms-excel':
                
                fullText = textScrap.getTextExcel(f"/Volumes/DD Thibault/sncf_DB/doc/{self.current_DB}/{file['KidFile']}")
                return fullText

            case 'application/vnd.openxmlformats-officedocument.wordprocessingml.document':
                fullText = textScrap.getTextWord(f"/Volumes/DD Thibault/sncf_DB/doc/{self.current_DB}/{file['KidFile']}")
                return fullText
            
            #case 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet':
                #fullText = textScrap.getTextWord(f"/Volumes/DD Thibault/sncf_DB/doc/engie/{file['KidFile']}")
                #return fullText
            #case 'image/jpeg':
                #return 2

            case _:
                return 'document type not supported'
                #raise ValueError('document type not supported')   # 0 is the default case if x is not found
    

In [3]:
csvPath = "/Volumes/DD Thibault/sncf_DB/doc/sncf_vinci_ifg_62.csv"
scrapper = TextScrapper()
csv = scrapper.getTextCsv(csvPath)
dictPdfWord=[]
fileManager = fileTextManager()
print(len(csv))
print(csv.iloc[4])
orderedCorpus = {}
dictType={}

for row in tqdm((csv.iloc)):
    if row["mimeType"] == "application/pdf" or row["mimeType"] == "application/vnd.openxmlformats-officedocument.wordprocessingml.document":
        contentFile = fileManager.getFileText(row)
        if row['KidDocumentClass'] in dictType :
            dictType[row['KidDocumentClass']].append(contentFile)
        else:
            dictType[row['KidDocumentClass']]=[]
            dictType[row['KidDocumentClass']].append(contentFile)
        #TODO charge content by batch
data = {}

for key in dictType:
    if len(dictType[key]) >= 20:
        if key in data:
            data[key]["data"], data[key]["test"]= train_test_split(dictType[key],test_size=0.20)
        else :
            data[key] = {"data":[], "test":[]}
            data[key]["data"], data[key]["test"]= train_test_split(dictType[key],test_size=0.20)
            


2470
fileName                   4129K-B006-GAZ.pdf
KidDocumentClass       K_NODE;cATERTETANCHGAZ
fileSize                               799744
mimeType                      application/pdf
owner               PROFIL;0001015M0324LV55BY
creationDate              2021-02-04 14:58:55
KidFile             K_NODE;0022805M2U19ARDWQT
Name: 4, dtype: object


2470it [06:58,  5.90it/s]

K_NODE;cATERTETANCHGAZ
K_NODE;cATERTETANCHGAZ
51
K_NODE;cATERTRAMON
K_NODE;cATERTRAMON
122
K_NODE;cATERTFLUIDO
K_NODE;cATERTFLUIDO
137
K_NODE;cATERTDISCO
K_NODE;cATERTDISCO
124
K_NODE;cATERTDEBAIR
K_NODE;cATERTDEBAIR
50
K_NODE;cATERDESENFNAT
K_NODE;cATERDESENFNAT
61
K_NODE;cATERTRIA
K_NODE;cATERTPOT
K_NODE;cATERTPOT
53
K_NODE;cATERTEXG
K_NODE;cATERTPAU
K_NODE;cATERTPAU
1756
K_NODE;cATERTSSI
K_NODE;cATERTSSI
91
K_NODE;cATERTATH


K_NODE;cATERTETANCHGAZ
K_NODE;cATERTRAMON
K_NODE;cATERTFLUIDO
K_NODE;cATERTDISCO
K_NODE;cATERTDEBAIR
K_NODE;cATERDESENFNAT
K_NODE;cATERTPOT
K_NODE;cATERTPAU
K_NODE;cATERTSSI


In [22]:
tokenizeDict = {}
for types in tqdm(data):
    for file in data[types]["data"]:
        if types in tokenizeDict:
            if isinstance(file, str) and file != "":
                sentTokenize = sent_tokenize(file)
                wordTokenize = []
                for sent in sentTokenize:
                    wordTokenize.append(word_tokenize(sent))
                tokenizeDict[types]["data"].append(wordTokenize)
        else:
            tokenizeDict[types]={"data":[], "test":[]}
            if isinstance(file, str) and file != "":
                sentTokenize = sent_tokenize(file)
                wordTokenize = []
                for sent in sentTokenize:
                    wordTokenize.append(word_tokenize(sent))
                tokenizeDict[types]["data"].append(wordTokenize)
                
    for file in data[types]["test"]:
        if types in tokenizeDict:
            if isinstance(file, str) and file != "":
                sentTokenize = sent_tokenize(file)
                wordTokenize = []
                for sent in sentTokenize:
                    wordTokenize.append(word_tokenize(sent))
                tokenizeDict[types]["test"].append(wordTokenize)
        
print(tokenizeDict["K_NODE;cATERTETANCHGAZ"]["test"][0])
del dictType


100%|████████████████████████████████████████████████████████████████████████████████| 9/9 [00:04<00:00,  2.12it/s]

[['rapport', 'dintervention', 'contrôle', 'de', 'létanchéité', 'du', 'rés', 'eau', 'gaz', 'n', 'date', 'du', 'rapport', 'n', 'de', 'fiche', 'client', 'sncf', 'site', 'nbbrigade', 'voie', 'adresse', 'malesherbes', 'gare', 'malesherbes', 'responsable', 'affaire', 'mustapha', 'khallouk', 'responsable', 'technique', 'certificat', 'd', 'étanchéité', 'je', 'soussigné', 'abdelouahid', 'daoudi', 'avoir', 'effectué', 'le', 'contrôle', 'de', 'létanchéité', 'du', 'réseau', 'gaz', 'entre', 'le', 'point', 'de', 'livraison', 'du', 'site', 'à', 'léquipement', 'de', 'production', '.'], ['statut', 'de', 'létanchéité', 'du', 'réseau', 'de', 'gaz', 'étanche', 'observation', 'cuve', 'pour', 'aiguille', 'non', 'étanche', 'signalé', 'à', 'm.guymar', 'technicien', 'vinci', 'facilities', 'nom', 'et', 'signature', 'signature', 'client', 'abdelouahid', 'daoudi', 'abdelouahid.daoudivinci', '-facilities.com']]


NameError: name 'dictType' is not defined

In [25]:
associativeDictionary = {}
for types in tqdm(tokenizeDict):
    print(types)
    for file in tokenizeDict[types]["data"]:
        if types in associativeDictionary:
            associativeDictionary[types]["data"].append(gensim.corpora.Dictionary(file))
        else:
            associativeDictionary[types]={"data":[], "test":[]}
            associativeDictionary[types]["data"].append(gensim.corpora.Dictionary(file))
    for file in tokenizeDict[types]["test"]:
        associativeDictionary[types]["test"].append(gensim.corpora.Dictionary(file))
        
print(associativeDictionary["K_NODE;cATERTETANCHGAZ"]["test"][8].token2id)



 33%|██████████████████████████▋                                                     | 3/9 [00:00<00:00, 17.74it/s]

K_NODE;cATERTETANCHGAZ
K_NODE;cATERTRAMON
K_NODE;cATERTFLUIDO
K_NODE;cATERTDISCO
K_NODE;cATERTDEBAIR
K_NODE;cATERDESENFNAT
K_NODE;cATERTPOT
K_NODE;cATERTPAU


100%|████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00,  9.92it/s]

K_NODE;cATERTSSI
{'-': 0, '.': 1, 'adresse': 2, 'affaire': 3, 'avenue': 4, 'avoir': 5, 'certificat': 6, 'client': 7, 'contrôle': 8, 'd': 9, 'date': 10, 'de': 11, 'dintervention': 12, 'du': 13, 'eau': 14, 'effectué': 15, 'entre': 16, 'fiche': 17, 'gaz': 18, 'je': 19, 'jeremy': 20, 'khallouk': 21, 'le': 22, 'livraison': 23, 'léquipement': 24, 'létanchéité': 25, 'mustapha': 26, 'n': 27, 'orleans': 28, 'paris': 29, 'point': 30, 'production': 31, 'puard': 32, 'rapport': 33, 'rbbatiment': 34, 'responsable': 35, 'rés': 36, 'réseau': 37, 'site': 38, 'sncf': 39, 'soussigné': 40, 'technique': 41, 'voyageurs': 42, 'à': 43, 'étanchéité': 44, '-facilities.com': 45, 'et': 46, 'facilities': 47, 'jeremy.puardvinci': 48, 'nom': 49, 'observation': 50, 'signature': 51, 'statut': 52, 'technicien': 53, 'vinci': 54, 'étanche': 55}


In [30]:
bowDict = {}
#TODO -----> remove flat_list because i think than next step has need list of list and not flat list


for types in tokenizeDict:
    print(types)
    i=0
    j=0
    for file in tokenizeDict[types]["data"]:
        tampFile = []
        for sentence in file:
            if types in bowDict:
                tampFile.append(associativeDictionary[types]["data"][i].doc2bow(sentence))
            else:
                bowDict[types]={"data":[], "test":[]}
                tampFile.append(associativeDictionary[types]["data"][i].doc2bow(sentence))
        bowDict[types]["data"].append(tampFile)
    for file in tokenizeDict[types]["test"]:
        tampFile = []
        for sentence in file:
            tampFile.append(associativeDictionary[types]["test"][j].doc2bow(sentence))
        bowDict[types]["test"].append(tampFile)
        
    i+=1
    j+=1
print(associativeDictionary["K_NODE;cATERTETANCHGAZ"]["data"][0].token2id)
print(bowDict["K_NODE;cATERTETANCHGAZ"]["data"][0])

{'.': 0, 'abdelouahid': 1, 'adresse': 2, 'affaire': 3, 'avoir': 4, 'certificat': 5, 'client': 6, 'contrôle': 7, 'd': 8, 'daoudi': 9, 'date': 10, 'de': 11, 'dintervention': 12, 'du': 13, 'eau': 14, 'effectué': 15, 'entre': 16, 'fiche': 17, 'gare': 18, 'gaz': 19, 'je': 20, 'khallouk': 21, 'le': 22, 'livraison': 23, 'léquipement': 24, 'létanchéité': 25, 'malesherbes': 26, 'mustapha': 27, 'n': 28, 'nbbrigade': 29, 'point': 30, 'production': 31, 'rapport': 32, 'responsable': 33, 'rés': 34, 'réseau': 35, 'site': 36, 'sncf': 37, 'soussigné': 38, 'technique': 39, 'voie': 40, 'à': 41, 'étanchéité': 42, '-facilities.com': 43, 'abdelouahid.daoudivinci': 44, 'aiguille': 45, 'cuve': 46, 'et': 47, 'facilities': 48, 'm.guymar': 49, 'nom': 50, 'non': 51, 'observation': 52, 'pour': 53, 'signalé': 54, 'signature': 55, 'statut': 56, 'technicien': 57, 'vinci': 58, 'étanche': 59}
K_NODE;cATERTETANCHGAZ
K_NODE;cATERTRAMON
K_NODE;cATERTFLUIDO
K_NODE;cATERTDISCO
K_NODE;cATERTDEBAIR
K_NODE;cATERDESENFNAT
K_NOD

In [34]:
tfidfDict= {}

for types in bowDict:

    for file in bowDict[types]["data"]:
        
        if types in tfidfDict:
            tfidfDict[types]["data"].append(gensim.models.TfidfModel(file))
        else:
            tfidfDict[types]={"data":[], "test":[]}
            tfidfDict[types]["data"].append(gensim.models.TfidfModel(file))
    
    for file in bowDict[types]["test"]:
        
        tfidfDict[types]["test"].append(gensim.models.TfidfModel(file))
        
print(tfidfDict["K_NODE;cATERTETANCHGAZ"]["test"][0])

TfidfModel<num_docs=2, num_nnz=69>


TfidfModel<num_docs=5, num_nnz=160>


In [67]:
simsDict = {}
bowTest= bowDict["K_NODE;cATERTETANCHGAZ"]["data"]
for types in tfidfDict: 
    j=0
    i=0
    print(types)
    for file in tfidfDict[types]["data"]:
        print(associativeDictionary[types]["data"][i])
        print("___________________________")
        if types in simsDict:
            simsDict[types]["data"].append(gensim.similarities.Similarity('workdir/',file[bowDict[types]["data"][i]],num_features=len(associativeDictionary[types]["data"][i])))
        else:
            simsDict[types] = {"data":[], "test":[]}
            tfidf = tfidfDict["K_NODE;cATERTDEBAIR"]["data"]
            simsDict[types]["data"].append(gensim.similarities.Similarity('workdir/',file[bowDict[types]["data"][i]],num_features=len(associativeDictionary[types]["data"][i])))
        i+=1
        
    for file in tfidfDict[types]["test"]:
        simsDict[types]["test"].append(gensim.similarities.Similarity('workdir/',file[bowTest[j]],num_features=len(associativeDictionary[types]["test"][j])))
        
        j+=1
print(simsDict["K_NODE;cATERTETANCHGAZ"]["test"][5])

K_NODE;cATERTETANCHGAZ
Dictionary<55 unique tokens: ['.', 'client', 'contrôle', 'date', 'de']...>
___________________________
Dictionary<75 unique tokens: ['-corps', '-des', '-pierre', '.', 'adresse']...>
___________________________
Dictionary<59 unique tokens: ['.', 'adresse', 'affaire', 'avoir', 'certificat']...>
___________________________
Dictionary<55 unique tokens: ['.', 'adresse', 'affaire', 'avoir', 'certificat']...>
___________________________
Dictionary<59 unique tokens: ['-aubin', '-saint', '.', 'abdelouahid', 'adresse']...>
___________________________
Dictionary<63 unique tokens: ['.', 'client', 'contrôle', 'date', 'de']...>
___________________________
Dictionary<58 unique tokens: ['.', 'adresse', 'affaire', 'avoir', 'bessaque']...>
___________________________
Dictionary<78 unique tokens: ['-corps', '-des', '-pierre', '.', 'adresse']...>
___________________________
Dictionary<60 unique tokens: ['.', 'client', 'contrôle', 'date', 'de']...>
___________________________
Diction

IndexError: index 50 is out of bounds for axis 0 with size 50

In [56]:
def setupFileToCompare(file):
    
    #documentTest = data["K_NODE;cATERTDISCO"]["test"][13]
    sents = sent_tokenize(file)
    wordTest=[]
    for sent in sents: 
        wordTest.append(word_tokenize(sent))
    dictDocTest = gensim.corpora.Dictionary(wordTest)
    bowDocTest = []
    for sentence in wordTest:
        bowDocTest.append(dictDocTest.doc2bow(sentence))
    return bowDocTest

bowFile = setupFileToCompare(data["K_NODE;cATERTDISCO"]["test"][13])
print(bowFile)

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 3), (8, 1), (9, 1), (10, 1), (11, 2), (12, 1), (13, 1), (14, 1), (15, 1), (16, 2), (17, 1), (18, 1), (19, 1), (20, 3), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 2), (27, 1), (28, 1), (29, 1), (30, 2), (31, 1), (32, 1), (33, 1), (34, 2)], [(0, 1), (2, 1), (7, 14), (8, 3), (9, 3), (11, 5), (13, 5), (14, 3), (15, 1), (19, 1), (20, 1), (22, 1), (24, 1), (26, 1), (28, 2), (29, 6), (33, 1), (34, 1), (35, 9), (36, 1), (37, 3), (38, 3), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 2), (46, 2), (47, 1), (48, 1), (49, 2), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 2), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 2), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 3), (76, 2), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 2), (91, 1), (92, 1), (93, 1), (9

In [58]:
#try to have a average about document types similaritys
simsList=[]
docSimilarity=[]
for types in simsDict:
    for sims in simsDict[types]["data"]:
        print(sims)
        print(sims[bowFile])
        break
    break

Similarity<3 documents in 0 shards stored under workdir/>


FileNotFoundError: [Errno 2] No such file or directory: 'workdir/.0'